In [1]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten, UpSampling2D, GlobalAveragePooling2D
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers.normalization import BatchNormalization
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.datasets import cifar100
from tensorflow.keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
from keras import backend as K

Using TensorFlow backend.


In [2]:
num_classes = 100

(x, y), (x_test, y_test) = cifar100.load_data()
# x_val, y_val = x_train[40000:50000,:], y_train[40000:50000]
# x_train, y_train = x_train[:40000,:], y_train[:40000]
# random data
# from sklearn.model_selection import train_test_split
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
#                                                     test_size=0.2,
#                                                     random_state=0,
#                                                     stratify=y_train)
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
st = StratifiedShuffleSplit(n_splits = 2, test_size = 0.2, random_state = 0)
for train_index, val_index in st.split(x, y):
    x_train, x_val, y_train, y_val = x[train_index], x[val_index], y[train_index], y[val_index]
    
print('random data = ',x_train.shape,x_val.shape,y_train.shape,y_val.shape)

random data =  (40000, 32, 32, 3) (10000, 32, 32, 3) (40000, 1) (10000, 1)


In [3]:
'''
Vẽ biểu đồ dataset
'''
n, bins, patches = plt.hist(x=y_val[:], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(x=y_train[:], bins='auto', color='#607c8e',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('y')
plt.ylabel('number')
plt.title('dataset')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
plt.show()

NameError: name 'y_val' is not defined

In [4]:

x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
x_val = x_val.reshape(x_val.shape[0], 32, 32, 3)
x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)

x_val = preprocess_input(x_val)
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

NameError: name 'x_val' is not defined

In [5]:
y_val = np_utils.to_categorical(y_val, num_classes)
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)


filepath = 'model.h5'

checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

# stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
stop =  EarlyStopping(monitor='val_accuracy', min_delta=0.001, 
                      verbose=0, mode='auto', baseline=None, 
                      restore_best_weights=False)


callbacks = [checkpoint, stop]

datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.1, height_shift_range=0.1,shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

datagen.fit(x_train)

NameError: name 'y_val' is not defined

In [6]:
resnet_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3), classes = 100)

for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

model = Sequential()
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

234553344/234545216 [==============================] - 31s 0us/step


In [7]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

historytemp = model.fit_generator(datagen.flow(x_train, y_train,
                                  batch_size=64),
                                  steps_per_epoch=x_train.shape[0] // 64,
                                  epochs=20,
                                  verbose=1,
                                  validation_data=(x_val, y_val),
                                  callbacks=callbacks)

NameError: name 'datagen' is not defined

In [8]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [9]:
score = model.evaluate(x_test, y_test, verbose=1)
print('[accuracy_test]: ',score)

# Check its architecture
# new_model.summary()
# x_test = preprocess_input(x_test)
# loss, acc = new_model.evaluate(x_test,  y_test, verbose=1)
# print('[accuracy_test]: {:5.2f}%'.format(100*acc))

ValueError: in user code:

    /home/mjnk9xw/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:941 test_function  *
        outputs = self.distribute_strategy.run(
    /home/mjnk9xw/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/mjnk9xw/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/mjnk9xw/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/mjnk9xw/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:912 test_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    /home/mjnk9xw/anaconda3/lib/python3.7/site-packages/tensorflo

In [10]:
import matplotlib.pyplot as plt
# list all data in history
print(historytemp.history.keys())
# summarize history for accuracy
plt.plot(historytemp.history['accuracy'])
plt.plot(historytemp.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(historytemp.history['loss'])
plt.plot(historytemp.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'historytemp' is not defined